In [ ]:
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.Models
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
from sklearn.model_selection import GroupKFold
from random import sample
from scipy import stats
from sklearn.metrics import mean_absolute_error
from collections import defaultdict
import time
from keras.applications.resnet50 import ResNet50
from sklearn.dummy import DummyRegressor
import random
from sklearn.preprocessing import QuantileTransformer

In [ ]:
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
from rpy2.robjects.numpy2ri import numpy2ri
import sys, getopt
import re
import tzlocal
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
from rpy2.robjects import pandas2ri

In [ ]:
#Model
folds = 10 #number of folds
features = 1 #number of features to predict
epochs = 200 #number of epochs
optimiser = 'adam' #model optimiser
loss = 'mse' #model loss
ntrees = 100
kernel_size = 2

In [ ]:
#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [ ]:
#Define the predictor datasets
omics = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/src/butterfly')

In [ ]:
with open('albums_all.pkl', 'rb') as f:
    albums_all = pickle.load(f)

In [ ]:
with open('albums.pkl', 'rb') as f:
    albums = pickle.load(f)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/notebooks')

In [ ]:
with open('RF_predictor.pkl', 'rb') as f:
    RF_predictor = pickle.load(f)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles')

In [ ]:
with open('groups_c.pkl', 'rb') as f:
    groups_c = pickle.load(f)

In [ ]:
with open('responses_50.pkl', 'rb') as f:
    responses = pickle.load(f)

In [ ]:
n_samples = 50 #same as the number of responses

In [ ]:
utils = importr('utils')

In [ ]:
utils.install_packages('pcLasso')

In [ ]:
groups = DF['patientID']

group_cols = groups_c[predictor_index]

#Get your response dataset
response = responses[predictor_index]
response_df = DF[response]
y = response_df.values

yy = y.copy()

y = QuantileTransformer().fit_transform(y)

y = y[:,feat_n]

X = RF_predictor[predictor_index]

#############

results = []

y_prediction_train = []
y_observed_train = []

y_prediction_test = []
y_observed_test = []

group_kfold = GroupKFold(n_splits=folds)

y_pred_train_models = []
y_pred_test_models = []

for train_index, test_index in group_kfold.split(X, y, groups):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    yy_train, yy_test = yy[train_index], yy[test_index]
    groups_train = groups[train_index]

    #Model 1
    RF = RandomForestRegressor(n_estimators=ntrees,
                               min_samples_split = 5,
                               random_state=0)
    RF.fit(X_train, y_train)

    #Model 2
    Lasso = linear_model.Lasso(alpha=0.1)
    Lasso.fit(X_train, y_train)

    #Model 3
    y_train_gl = y_train.reshape(-1, 1)

    sparsegroupLasso = GroupLasso(
        groups=group_cols,
        group_reg=5,
        l1_reg=0,
        frobenius_lipschitz=True,
        scale_reg="inverse_group_size",
        subsampling_scheme=1,
        supress_warning=True,
        n_iter=1000,
        tol=1e-3,
        )
    sparsegroupLasso.fit(X_train, y_train_gl)

    # demonstrate prediction        
    y_pred_train_RF = pd.DataFrame(RF.predict(X_train))
    y_pred_train_ls = pd.DataFrame(Lasso.predict(X_train))
    y_pred_train_gl = pd.DataFrame(sparsegroupLasso.predict(X_train))

    y_pred_train_models.append(y_pred_train_RF)
    y_pred_train_models.append(y_pred_train_ls)
    y_pred_train_models.append(y_pred_train_gl)

    y_pred_test_RF = pd.DataFrame(RF.predict(X_test))
    y_pred_test_ls = pd.DataFrame(Lasso.predict(X_test))
    y_pred_test_gl = pd.DataFrame(sparsegroupLasso.predict(X_test))

    y_pred_test_models.append(y_pred_test_RF)
    y_pred_test_models.append(y_pred_test_ls)
    y_pred_test_models.append(y_pred_test_gl)

    y_pred_test_train_all = []
    y_obsr_test_train = []

    for train_train_index, test_train_index in group_kfold.split(X_train, y_train, groups_train):

        X_train_train, X_test_train = X_train[train_train_index], X_train[test_train_index]
        y_train_train, y_test_train = y_train[train_train_index], y_train[test_train_index]
        yy_train_train, yy_test_train = yy_train[train_train_index], yy_train[test_train_index]

        y_pred_test_train = []

        #Model 1
        RF = RandomForestRegressor(n_estimators=ntrees,
                                   min_samples_split = 5,
                                   random_state=0)
        RF.fit(X_train_train, y_train_train)

        #Model 2
        Lasso = linear_model.Lasso(alpha=0.1)
        Lasso.fit(X_train_train, y_train_train)

        #Model 3
        y_train_train_gl = y_train_train.reshape(-1, 1)

        sparsegroupLasso = GroupLasso(
            groups=group_cols,
            group_reg=5,
            l1_reg=0,
            frobenius_lipschitz=True,
            scale_reg="inverse_group_size",
            subsampling_scheme=1,
            supress_warning=True,
            n_iter=1000,
            tol=1e-3,
            )
        sparsegroupLasso.fit(X_train_train, y_train_train_gl)

        RF_pred = RF.predict(X_test_train)
        LS_pred = Lasso.predict(X_test_train)
        GL_pred = sparsegroupLasso.predict(X_test_train)

        y_pred_test_train.append(RF_pred)        
        y_pred_test_train.append(LS_pred)
        y_pred_test_train.append(GL_pred)

        y_pred_test_train_cv = pd.DataFrame(y_pred_test_train).transpose() 
        y_pred_test_train_all.append(y_pred_test_train_cv)
        y_obsr_test_train.append(y_test_train)

    y_pred_all_train = pd.concat(y_pred_test_train_all)
    y_obsr_all_train = np.concatenate(y_obsr_test_train, axis=None)

    if metaclassifier == "non_negative":

        meta_model = linear_model.Lasso(alpha=0.1, positive = True)

    meta_model.fit(y_pred_all_train, y_obsr_all_train)


    y_pred_train = pd.DataFrame(y_pred_train_models).transpose() 
    X_tr = y_pred_train.values                

    y_pred_test = pd.DataFrame(y_pred_test_models).transpose() 
    X_tst = y_pred_test.values                

In [ ]:
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.Stacked_models)(DF, responses, predictor_index, 
                                                              feat_n, RF_predictor, folds, ntrees,
                                                              "matrix", groups_c, "non_negative", False)
                            for feat_n in range(n_samples)))
        
    RF['prediction_train'].append(prediction_train)
    RF['observed_train'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_50')
with open('Stacked_models_NS.pkl', 'wb') as f:  
    pickle.dump(RF, f)

In [ ]:
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.RLRF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'pcLasso', 'matrix', groups_c, True)for feat_n in list(range(20, 22))))
        
    RF['prediction_train'].append(prediction_train)
    RF['observed_train'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_50')
with open('pcLasso_M_S_11.pkl', 'wb') as f:  
    pickle.dump(RF, f)

In [ ]:
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.RLRF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'blockForest', 'matrix', groups_c, False)
                            for feat_n in range(0, n_samples)))
        
    RF['prediction_train'].append(prediction_train)
    RF['observed_train'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_50')
with open('BRF_M_NS.pkl', 'wb') as f:  
    pickle.dump(RF, f)

In [ ]:
DNN = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):
        
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.RLRF)(DF, responses, predictor_index, feat_n, RF_predictor, 
                      folds, ntrees, "pcLasso", 'matrix', groups_c, True)
                            for feat_n in range(n_samples)))
        
    DNN['prediction_train'].append(prediction_train)
    DNN['observed_train'].append(observed_train)
    DNN['prediction_test'].append(prediction_test)
    DNN['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_20_new4')
with open('DNN_M_NS.pkl', 'wb') as f:  
    pickle.dump(DNN, f)

In [ ]:
DNN = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):
        
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.NN)(RF_predictor, DF, feat_n, predictor_index,  
                            responses, 128, features, folds, epochs, optimiser, loss, 
                                                          'DNN', 'matrix_2', 2, False)
                            for feat_n in range(n_samples)))
        
    DNN['prediction_train'].append(prediction_train)
    DNN['observed_train'].append(observed_train)
    DNN['prediction_test'].append(prediction_test)
    DNN['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_20_new4')
with open('DNN_M_NS.pkl', 'wb') as f:  
    pickle.dump(DNN, f)

In [ ]:
CNN = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):
        
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.CNN)(RF_predictor, DF, feat_n, predictor_index,  
                            responses, 128, features, folds, epochs, optimiser, loss, 'CNN', 
                                                          'matrix_3', 1)for feat_n in range(n_samples)))
        
    CNN['prediction_train'].append(prediction_train)
    CNN['observed_train'].append(observed_train)
    CNN['prediction_test'].append(prediction_test)
    CNN['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_20_new3')
with open('CNN_M_S2.pkl', 'wb') as f:  
    pickle.dump(CNN, f)

In [ ]:
MCNN = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):
        
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.CNN)(albums, DF, feat_n, predictor_index,  
                            responses, 40, features, folds, epochs, optimiser, loss, 'MCNN',
                                                         'TSNE_M',2)for feat_n in range(n_samples)))
        
    MCNN['prediction_train'].append(prediction_train)
    MCNN['observed_train'].append(observed_train)
    MCNN['prediction_test'].append(prediction_test)
    MCNN['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_20_new3')
with open('MCNN_T_NS.pkl', 'wb') as f:  
    pickle.dump(MCNN, f)

In [ ]:
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.LRF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'RF_regressor', 'matrix', groups_c, True)for feat_n in range(n_samples)))
        
    RF['prediction_train'].append(prediction_train)
    RF['observed_train'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_50')
with open('RF_M_S.pkl', 'wb') as f:  
    pickle.dump(RF, f)

In [ ]:
LS = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.RF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'Lasso', 'matrix')for feat_n in range(n_samples)))
        
    LS['prediction_train'].append(prediction_train)
    LS['observed_train'].append(observed_train)
    LS['prediction_test'].append(prediction_test)
    LS['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles/models_20_new2')
with open('LS_S.pkl', 'wb') as f:  
    pickle.dump(LS, f)

In [ ]:
Dummy = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.RF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'Dummy')for feat_n in range(n_samples)))
        
    Dummy['prediction_train'].append(prediction_train)
    Dummy['observed_train'].append(observed_train)
    Dummy['prediction_test'].append(prediction_test)
    Dummy['observed_test'].append(observed_test)

In [ ]:
with open('Dummy.pkl', 'wb') as f:  
    pickle.dump(Dummy, f)

In [ ]:
# load model
model = ResNet50()
# summarize the model
#model.summary()

In [ ]:
n_samples = 30 #same as the number of responses

RN = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):
        
    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.CNN)(albums_all, DF, feat_n, predictor_index,  
                            responses, 128, features, folds, epochs, optimiser, loss, 'ResNet')for feat_n in range(n_samples)))
        
    RN['prediction_train'].append(prediction_train)
    RN['observed_train'].append(observed_train)
    RN['prediction_test'].append(prediction_test)
    RN['observed_test'].append(observed_test)

In [ ]:
os.chdir('/home/mxenoc/workspace/butterfly/notebooks')

In [ ]:
with open('MCNN_M.pkl', 'rb') as f:
    MCNN_M = pickle.load(f)

In [ ]:
with open('CNN_Matrix.pkl', 'rb') as f:
    CNN_Matrix = pickle.load(f)

In [ ]:
with open('CNN.pkl', 'rb') as f:
    CNN = pickle.load(f)

In [ ]:
with open('RF.pkl', 'rb') as f:
    RF = pickle.load(f)

In [ ]:
with open('MCNN.pkl', 'rb') as f:
    MCNN = pickle.load(f)

In [ ]:
with open('RF_T.pkl', 'rb') as f:
    RF_T = pickle.load(f)

In [ ]:
#responses
feats  = np.hstack([np.tile('rna',n_samples), np.tile('plasma_l',n_samples), np.tile('serum_l',n_samples), np.tile('microbiome',n_samples), 
                    np.tile('immune',n_samples), np.tile('metabolomics',n_samples), np.tile('plasma_s',n_samples)])

In [ ]:
each_omic_CNN_r = []
each_omic_CNN_p = []

all_features_CNN_r = []
all_features_CNN_p = []

for l in range(len(omics)):
    each_feature_CNN_r = []
    each_feature_CNN_p = []
    for k in range(n_samples):
            each_feature_CNN_r.append(stats.spearmanr(CNN['prediction_test'][l][k], 
                                                          CNN['observed_test'][l][k])[0])
            each_feature_CNN_p.append(stats.spearmanr(CNN['prediction_test'][l][k], 
                                                          CNN['observed_test'][l][k])[1])
    all_features_CNN_r.append(each_feature_CNN_r)
    all_features_CNN_p.append(each_feature_CNN_p)
    each_omic_CNN_r.append(np.mean(each_feature_CNN_r))    
    each_omic_CNN_p.append(np.mean(each_feature_CNN_p))    
    
CNN_r = np.hstack(all_features_CNN_r)
CNN_p = np.hstack(all_features_CNN_p)
CNN_r_p = pd.DataFrame({'r': CNN_r, 'p': CNN_p, 'omics': feats, 'model': 'CNN'})

In [ ]:
CNN_r_p = pd.DataFrame({'r': CNN_r, 'p': CNN_p, 'omics': feats, 'model': 'CNN'})

In [ ]:
each_omic_LS_r = []
each_omic_LS_p = []

all_features_LS_r = []
all_features_LS_p = []

for l in range(len(omics)):
    each_feature_LS_r = []
    each_feature_LS_p = []
    for k in range(n_samples):
            each_feature_LS_r.append(stats.spearmanr(LS['prediction_test'][l][k], 
                                                          LS['observed_test'][l][k])[0])
            each_feature_LS_p.append(stats.spearmanr(LS['prediction_test'][l][k], 
                                                          LS['observed_test'][l][k])[1])
    all_features_LS_r.append(each_feature_LS_r)
    all_features_LS_p.append(each_feature_LS_p)
    each_omic_LS_r.append(np.mean(each_feature_LS_r))    
    each_omic_LS_p.append(np.mean(each_feature_LS_p))    
    
LS_r = np.hstack(all_features_LS_r)
LS_p = np.hstack(all_features_LS_p)
LS_r_p = pd.DataFrame({'r': LS_r, 'p': LS_p, 'omics': feats, 'model': 'LS'})

In [ ]:
LS_r_p = pd.DataFrame({'r': LS_r, 'p': LS_p, 'omics': feats, 'model': 'LS'})

In [ ]:
each_omic_r = []
each_omic_p = []

all_features_r = []
all_features_p = []

for l in range(len(omics)):
    each_feature_r = []
    each_feature_p = []
    for k in range(n_samples):
            each_feature_r.append(stats.spearmanr(MCNN_M['prediction_test'][l][k], 
                                                          MCNN_M['observed_test'][l][k])[0])
            each_feature_p.append(stats.spearmanr(MCNN_M['prediction_test'][l][k], 
                                                          MCNN_M['observed_test'][l][k])[1])
    all_features_r.append(each_feature_r)
    all_features_p.append(each_feature_p)
    each_omic_r.append(np.mean(each_feature_r))    
    each_omic_p.append(np.mean(each_feature_p))  
    
r = np.hstack(all_features_r)
p = np.hstack(all_features_p)
MCNN_M_r_p = pd.DataFrame({'r': r, 'p': p, 'omics': feats, 'model': 'CNN_Matrix'})

In [ ]:
each_omic_RF_r = []
each_omic_RF_p = []

all_features_RF_r = []
all_features_RF_p = []

for l in range(len(omics)):
    each_feature_RF_r = []
    each_feature_RF_p = []
    for k in range(n_samples):
            each_feature_RF_r.append(stats.spearmanr(RF['prediction_test'][l][k], 
                                                          RF['observed_test'][l][k])[0])
            each_feature_RF_p.append(stats.spearmanr(RF['prediction_test'][l][k], 
                                                          RF['observed_test'][l][k])[1])
    all_features_RF_r.append(each_feature_RF_r)
    all_features_RF_p.append(each_feature_RF_p)
    each_omic_RF_r.append(np.mean(each_feature_RF_r))    
    each_omic_RF_p.append(np.mean(each_feature_RF_p))    
    
RF_r = np.hstack(all_features_RF_r)
RF_p = np.hstack(all_features_RF_p)
RF_r_p = pd.DataFrame({'r': RF_r, 'p': RF_p, 'omics': feats, 'model': 'RF'})

In [ ]:
RF_r_p = pd.DataFrame({'r': RF_r, 'p': RF_p, 'omics': feats, 'model': 'RF'})

In [ ]:
each_omic_MCNN_r = []
each_omic_MCNN_p = []

all_features_MCNN_r = []
all_features_MCNN_p = []

for l in range(len(omics)):
    each_feature_MCNN_r = []
    each_feature_MCNN_p = []
    for k in range(n_samples):
            each_feature_MCNN_r.append(stats.spearmanr(MCNN['prediction_test'][l][k], 
                                                          MCNN['observed_test'][l][k])[0])
            each_feature_MCNN_p.append(stats.spearmanr(MCNN['prediction_test'][l][k], 
                                                          MCNN['observed_test'][l][k])[1])
    all_features_MCNN_r.append(each_feature_MCNN_r)
    all_features_MCNN_p.append(each_feature_MCNN_p)
    each_omic_MCNN_r.append(np.mean(each_feature_MCNN_r))    
    each_omic_MCNN_p.append(np.mean(each_feature_MCNN_p))    
    
MCNN_r = np.hstack(all_features_MCNN_r)
MCNN_p = np.hstack(all_features_MCNN_p)
MCNN_r_p = pd.DataFrame({'r': MCNN_r, 'p': MCNN_p, 'omics': feats, 'model': 'MCNN'})

In [ ]:
MCNN_r_p = pd.DataFrame({'r': MCNN_r, 'p': MCNN_p, 'omics': feats, 'model': 'MCNN'})

In [ ]:
each_omic_Dummy_r = []
each_omic_Dummy_p = []

all_features_Dummy_r = []
all_features_Dummy_p = []

for l in range(len(omics)):
    each_feature_Dummy_r = []
    each_feature_Dummy_p = []
    for k in range(n_samples):
            each_feature_Dummy_r.append(stats.spearmanr(Dummy['prediction_test'][l][k], 
                                                          Dummy['observed_test'][l][k])[0])
            each_feature_Dummy_p.append(stats.spearmanr(Dummy['prediction_test'][l][k], 
                                                          Dummy['observed_test'][l][k])[1])
    all_features_Dummy_r.append(each_feature_Dummy_r)
    all_features_Dummy_p.append(each_feature_Dummy_p)
    each_omic_Dummy_r.append(np.mean(each_feature_Dummy_r))    
    each_omic_Dummy_p.append(np.mean(each_feature_Dummy_p))    
    
Dummy_r = np.hstack(all_features_Dummy_r)
Dummy_p = np.hstack(all_features_Dummy_p)
Dummy_r_p = pd.DataFrame({'r': Dummy_r, 'p': Dummy_p, 'omics': feats, 'model': 'Dummy'})

In [ ]:
Dummy_r_p = pd.DataFrame({'r': Dummy_r, 'p': Dummy_p, 'omics': feats, 'model': 'Dummy'})

In [ ]:
each_omic_Random_r = []
each_omic_Random_p = []

all_features_Random_r = []
all_features_Random_p = []

for l in range(len(omics)):
    each_feature_Random_r = []
    each_feature_Random_p = []
    for k in range(n_samples):
            each_feature_Random_r.append(stats.spearmanr(pd.DataFrame(random.sample(range(10, 100), 68)), 
                                                          RF['observed_test'][l][k])[0])
            each_feature_Random_p.append(stats.spearmanr(pd.DataFrame(random.sample(range(10, 100), 68)), 
                                                          RF['observed_test'][l][k])[1])
    all_features_Random_r.append(each_feature_Random_r)
    all_features_Random_p.append(each_feature_Random_p)
    each_omic_Random_r.append(np.mean(each_feature_Random_r))    
    each_omic_Random_p.append(np.mean(each_feature_Random_p))    
    
Random_r = np.hstack(all_features_Random_r)
Random_p = np.hstack(all_features_Random_p)
Random_r_p = pd.DataFrame({'r': Random_r, 'p': Random_p, 'omics': feats, 'model': 'Random'})

In [ ]:
Random_r_p = pd.DataFrame({'r': Random_r, 'p': Random_p, 'omics': feats, 'model': 'Random'})

In [ ]:
Random_df = Random_r_p[(Random_r_p['r']>0) & (Random_r_p['p']<=0.05)]

In [ ]:
Dummy_df = Dummy_r_p[(Dummy_r_p['r']>0) & (Dummy_r_p['p']<=0.05)]

In [ ]:
CNN_Matrix_df = CNN_Matrix_r_p[(CNN_Matrix_r_p['r']>0) & (CNN_Matrix_r_p['p']<=0.05)]

In [ ]:
MCNN_df = MCNN_r_p[(MCNN_r_p['r']>0) & (MCNN_r_p['p']<=0.05)]

In [ ]:
RF_T_df = RF_T_r_p[(RF_T_r_p['r']>0) & (RF_T_r_p['p']<=0.05)]

In [ ]:
RF_df = RF_r_p[(RF_r_p['r']>0) & (RF_r_p['p']<=0.05)]

In [ ]:
LS_df = LS_r_p[(LS_r_p['r']>0) & (LS_r_p['p']<=0.05)]

In [ ]:
CNN_df = CNN_r_p[(CNN_r_p['r']>0) & (CNN_r_p['p']<=0.05)]

In [ ]:
df_frames = [RF_df, RF_T_df, LS_df, CNN_df, CNN_Matrix_df, MCNN_df, Random_df]

In [ ]:
dfs = pd.concat(df_frames, sort=True)

In [ ]:
dfs = dfs.iloc[:,0:2]

In [ ]:
dfs = dfs.groupby(["omics", "model"]).size().reset_index(name="Time")

In [ ]:
pivot_df = dfs.pivot(index='model', columns='omics', values='Time')

In [ ]:
pivot_df

In [ ]:
with open('pivot_df_60.pkl', 'wb') as f:  
    pickle.dump(pivot_df, f)

In [ ]:
colors = ["pink", "bisque","olive","skyblue","goldenrod","thistle","gray"]

In [ ]:
predictors = ['CNN_T', 'CNN_M', 'LASSO_M', 'MCNN_T', 'RF_M', 'RF_T', 'Random']
y_pos = np.arange(len(predictors))

In [ ]:
pivot_df.plot.bar(stacked=True, color=colors, figsize=(13,10))
plt.xticks(y_pos, predictors, rotation=0, fontsize=16)
plt.yticks(fontsize=16)
#plt.xlabel('Models', fontsize=18)
plt.legend(loc=2, prop={'size': 16})
plt.xlabel('')
plt.ylabel('Number of features predicted', fontsize=20)
plt.title('Prediction for 60 features from each dataset', fontsize=20)
plt.savefig('benchmark.png', dpi=500)

In [ ]:
predictors = ['RF', 'LASSO', 'CNN', 'MCNN', 'Random', 'Dummy']
y_pos = np.arange(len(predictors))

In [ ]:
features = [RF_df.shape[0], LS_df.shape[0], CNN_df.shape[0], 
            MCNN_df.shape[0], Random_df.shape[0], Dummy_df.shape[0]]

In [ ]:
import seaborn as sns

In [ ]:
#Plot the results
f = plt.figure()
plt.bar(y_pos, features, color=['firebrick', 'gold', 'olivedrab', 'royalblue', 
                                       'cyan', 'salmon'])
plt.xticks(y_pos, predictors)
plt.xlabel('Model')
plt.ylabel('Number of features predicted')
plt.title('Prediction for 20 features from each dataset')
plt.xticks(rotation=0)
#plt.show()

#os.chdir(Desktop)
f.savefig("Bars.pdf", bbox_inches='tight')

In [ ]:
#Boxplots
bxp = sns.boxplot(x=predictors, y=features, palette="Set2")
bxp.set_xticklabels(bxp.get_xticklabels(),rotation=0)
plt.xlabel('Models')
plt.ylabel('Spearman r')
plt.title('Accuracy over 20 features per omic dataset')
#plt.show()
fig = bxp.get_figure()
fig.savefig("boxesFeat_serum_l_test.pdf")